In [1]:
import cv2
import numpy as np
import time
import os
from picamera2 import Picamera2
import matplotlib.pyplot as plt 
from IPython.display import display, Image
import ipywidgets as widgets
import threading
import multiprocessing as mp
from collections import deque
#from show import *
from utils.ReadRadar import *
from utils.tracking import *
from utils.utils import *
from realtime_show import *

/home/heubuchm/devel/milliEye/.venv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# Stop button
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # (FontAwesome names without the `fa-` prefix)
)

0


In [3]:
# Display function
# ================
def view(button):
    display_handle=display(None, display_id=True)

    # Camera Initialization
    #video_fps = 30
    #cap = cv2.VideoCapture(0)
    #cap.set(3, 640)	
    #cap.set(4, 480)
    #cap.set(5, video_fps)

    picam2 = Picamera2()
    picam2.configure(picam2.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
    picam2.start()

    # Radar Initialization
    radar = readradar('./cfg/gim.cfg')

    # Configurate the serial port
    CLIport, Dataport = {}, {} 
    CLIport, Dataport = serialConfig(radar.configFileName)

    # Get the configuration parameters from the configuration file
    configParameters = parseConfigFile(radar.configFileName)

    detObj = {}  
    points_buffer = deque()   

    while True:
        time.sleep(1)
        time_start=time.time()

        # read video frames, it is a block operation
        #ret, frame = cap.read()
        frame = picam2.capture_array()

        # read radar frames
        dataOk = 0

        dataOk, frameNumber, detObj, timestamp = radar.readAndParseData68xx(Dataport, configParameters)
        print(dataOk)

        # check if there is data
        if dataOk:
            print('frameNumber: ', frameNumber, ' Detected Objects: ', detObj["numObj"])
            x, y, z = detObj["x"], detObj["y"], detObj["z"]
            points_3d = np.array([x, y, z])

            # projection
            calib_param = load_calib("./yaml/calibration.yaml")
            #points_2d = projection(points_3d, calib_param)

            # show
            #for i in points_2d:
            #    cv2.circle(frame, (i[0], i[1]), 7, (0,0,255), -1)
            #    display_handle.update(Image(data=frame.tobytes()))
                #cv2.imshow("capture", frame)
                #plt.imshow(image)
        _, frame = cv2.imencode('.png', frame)
        display_handle.update(Image(data=frame.tobytes()))

        time_end = time.time()
        #print('time cost', time_end-time_start,'s') 

        if stopButton.value==True:
            #cap.release()
            picam2.stop()
            picam2.close()
            display_handle.update(None)

In [4]:
# Run
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()

ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None

[0:28:07.031544203] [3196]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:28:07.100649193] [3197]  WARN RPiSdn sdn.cpp:39 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:28:07.104474300] [3197]  INFO RPI vc4.cpp:444 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media3 and ISP device /dev/media0
[0:28:07.104563352] [3197]  INFO RPI pipeline_base.cpp:1142 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[0:28:07.113111938] [3196]  INFO Camera camera_manager.cpp:284 libcamera v0.1.0+118-563cd78e
[0:28:07.164604315] [3200]  WARN RPiSdn sdn.cpp:39 Using legacy SDN tuning - please consider moving SDN inside rpi.denoise
[0:28:07.167287263] [3200]  INFO RPI vc4.cpp:444 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media3 and ISP device /dev/media0
[0:28:07.167349150] [3200]  INFO RPI pipeline_base.cpp:1142 Using configuration file '/usr/share/libcamera/pipeline

[0:28:07.180839909] [3196]  INFO Camera camera.cpp:1183 configuring streams: (0) 640x480-XRGB8888 (1) 640x480-SGBRG10_CSI2P
[0:28:07.181411442] [3200]  INFO RPI vc4.cpp:608 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 640x480-SGBRG10_1X10 - Selected unicam format: 640x480-pGAA


sensorStop
flushCfg
dfeDataOutputMode 1
channelCfg 15 7 0
adcCfg 2 1
adcbufCfg -1 0 1 1 1
profileCfg 0 60 761 7 40 0 0 100 1 64 2000 0 0 158
chirpCfg 0 0 0 0 0 0 0 1
chirpCfg 1 1 0 0 0 0 0 2
chirpCfg 2 2 0 0 0 0 0 4
frameCfg 0 2 16 0 200 1 0
lowPower 0 0
guiMonitor -1 1 1 0 0 0 1
cfarCfg -1 0 2 8 4 3 0 0 0
cfarCfg -1 1 0 4 2 3 1 5 1
multiObjBeamForming -1 1 0.5
clutterRemoval -1 0
calibDcRangeSig -1 0 -5 8 256
extendedMaxVelocity -1 0
lvdsStreamCfg -1 0 0 0
compRangeBiasAndRxChanPhase 0 1 0 -1 0 1 0 -1 0 1 0 -1 0 1 0 -1 0 1 0 -1 0 1 0 -1 0
measureRangeBiasAndRxChanPhase 0 1.5 0.2
CQRxSatMonitor 0 3 4 99 0
CQSigImgMonitor 0 31 4
analogMonitor 0 0
aoaFovCfg -1 -90 90 -20 90
cfarFovCfg -1 0 0.2 1.5
cfarFovCfg -1 1 -0.52 0.52
calibData 0 0 0
sensorStart
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
